<a href="https://colab.research.google.com/github/sayakpaul/AdaMatch-TF/blob/main/Vanilla_WideResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

## Datasets

* Source: MNIST
* Target: SVHN

In [2]:
(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = keras.datasets.mnist.load_data()

# Add a channel dimension
mnist_x_train = tf.expand_dims(mnist_x_train, -1)
mnist_x_test = tf.expand_dims(mnist_x_test, -1)

In [3]:
svhn_train, svhn_test = tfds.load("svhn_cropped", split=["train", "test"], as_supervised=True)

## Constants & hyperparameters

In [4]:
RESIZE_TO = 32
BATCH_SIZE = 64
EPOCHS = 10

AUTO = tf.data.AUTOTUNE
LEARNING_RATE = 0.03

## Data loaders

In [5]:
def augment(image, label, train=True):
    image = tf.image.resize_with_pad(image, RESIZE_TO, RESIZE_TO)
    image = tf.tile(image, [1, 1, 3])
    if train:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_crop(image, (RESIZE_TO, RESIZE_TO, 3))
    return (image, label)

In [6]:
source_train_ds = tf.data.Dataset.from_tensor_slices((mnist_x_train, mnist_y_train))
source_train_ds = (
    source_train_ds
    .shuffle(BATCH_SIZE * 100)
    .map(augment, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

source_test_ds = tf.data.Dataset.from_tensor_slices((mnist_x_test, mnist_y_test))
source_test_ds = (
    source_test_ds
    .map(lambda x, y: (augment(x, y, False)), num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

## Model, optimizer, and LR schedule

In [7]:
!wget -O wide_resnet.py -q https://git.io/Jnzzj
import wide_resnet

In [8]:
wrn_model = wide_resnet.get_network()
print(f"Model has {wrn_model.count_params()/1e6} Million parameters.")

Model has 1.471226 Million parameters.


In [10]:
STEPS_PER_EPOCH = len(mnist_x_train)//BATCH_SIZE
TOTAL_STEPS = EPOCHS * STEPS_PER_EPOCH

reduce_lr = keras.optimizers.schedules.CosineDecay(
    LEARNING_RATE, TOTAL_STEPS, 0.25)
optimizer = keras.optimizers.Adam(reduce_lr)

## Model training and evaluation

In [11]:
wrn_model.compile(optimizer=optimizer, metrics=["accuracy"], 
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True))
wrn_model.fit(source_train_ds, epochs=EPOCHS)

_, accuracy = wrn_model.evaluate(source_test_ds)
print(f"Accuracy on source test set: {accuracy * 100:.2f}%")

Epoch 1/10
938/938 [==============================] - 25s 23ms/step - loss: 1.1605 - accuracy: 0.8105
Epoch 2/10
938/938 [==============================] - 22s 23ms/step - loss: 0.5397 - accuracy: 0.9072
Epoch 3/10
938/938 [==============================] - 22s 23ms/step - loss: 0.4541 - accuracy: 0.9208
Epoch 4/10
938/938 [==============================] - 22s 23ms/step - loss: 0.4103 - accuracy: 0.9269
Epoch 5/10
938/938 [==============================] - 22s 23ms/step - loss: 0.3656 - accuracy: 0.9342
Epoch 6/10
938/938 [==============================] - 22s 23ms/step - loss: 0.3311 - accuracy: 0.9389
Epoch 7/10
938/938 [==============================] - 22s 23ms/step - loss: 0.2986 - accuracy: 0.9433
Epoch 8/10
938/938 [==============================] - 22s 23ms/step - loss: 0.2704 - accuracy: 0.9482
Epoch 9/10
938/938 [==============================] - 22s 23ms/step - loss: 0.2414 - accuracy: 0.9535
Epoch 10/10
157/157 [==============================] - 2s 8ms/step - loss: 1.4517 

In [12]:
svhn_test = svhn_test.batch(BATCH_SIZE).prefetch(AUTO)
_, accuracy = wrn_model.evaluate(svhn_test)
print(f"Accuracy on target test set: {accuracy * 100:.2f}%")

407/407 [==============================] - 4s 8ms/step - loss: 12.9453 - accuracy: 0.0720
Accuracy on target test set: 7.20%
